In [1]:
# %pip install tensorflow

In [2]:
# %pip install keras-tuner

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

train_features = pd.read_csv("./train_NN.csv")
train_labels = pd.read_csv("./train_labels_NN.csv")
test_features = pd.read_csv("test_NN.csv")

In [4]:
train_labels.head()
label_frequencies = train_labels['0'].value_counts()
print(label_frequencies)

1.0    88304
0.0    88304
Name: 0, dtype: int64


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels)

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Custom RBF activation function
def rbf_activation(x):
    return tf.math.exp(-1.0 * tf.square(x))

class SimpleNeuralNetwork:
    def __init__(self, input_dim, hidden_units, output_dim):
        self.model = self.build_model(input_dim, hidden_units, output_dim)

    def build_model(self, input_dim, hidden_units, output_dim):
        model = Sequential()
        model.add(Dense(hidden_units, input_dim=input_dim, activation='relu'))
        model.add(Dense(hidden_units, activation='relu'))
        model.add(Dense(output_dim, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    def train(self, X_train, y_train, epochs, batch_size, validation_data=None):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    def evaluate(self, X_test, y_test):
        loss, accuracy = self.model.evaluate(X_test, y_test)
        print(f'Test Loss: {loss:.4f}')
        print(f'Test Accuracy: {accuracy:.4f}')

    def predict(self, X_data, threshold=0.5):
        predictions_proba = self.model.predict(X_data)
        predictions = (predictions_proba > threshold).astype(int)
        return predictions

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from kerastuner.tuners import RandomSearch

# Custom RBF activation function
def rbf_activation(x):
    return tf.math.exp(-1.0 * tf.square(x))

class SimpleNeuralNetwork:
    def __init__(self, input_dim, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim

    def build_model(self, hp):
        model = Sequential()

        # Tune the number of hidden layers and units in each layer
        for i in range(hp.Int('num_hidden_layers', min_value=1, max_value=3)):
            model.add(Dense(units=hp.Int(f'units_{i}', min_value=8, max_value=128), 
                            activation=hp.Choice(f'activation_{i}', values=['relu', 'tanh']),
                            input_dim=self.input_dim if i == 0 else None))

        model.add(Dense(units=self.output_dim, activation='sigmoid'))

        # Tune the learning rate
        model.compile(
            loss='binary_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
            metrics=['accuracy']
        )

        return model

    def tune_hyperparameters(self, X_train, y_train, epochs, batch_size, validation_data=None):
        tuner = RandomSearch(
            self.build_model,
            objective='val_accuracy',
            max_trials=10,  # Number of hyperparameter combinations to try
            directory='keras_tuner_dir',  # Directory to store the results
            project_name='simple_neural_network'
        )

        tuner.search(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)
        
        # Get the best model and its hyperparameters
        best_model = tuner.get_best_models(num_models=1)[0]
        best_hyperparameters = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

        self.model = best_model
        return best_hyperparameters

    def train(self, X_train, y_train, epochs, batch_size, validation_data=None):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    def evaluate(self, X_test, y_test):
        loss, accuracy = self.model.evaluate(X_test, y_test)
        print(f'Test Loss: {loss:.4f}')
        print(f'Test Accuracy: {accuracy:.4f}')

    def predict(self, X_data, threshold=0.5):
        predictions_proba = self.model.predict(X_data)
        predictions = (predictions_proba > threshold).astype(int)
        return predictions


In [8]:
# Example usage
input_dim = X_train.shape[1]
hidden_units = input_dim
output_dim = 1

# Instantiate the SimpleNeuralNetwork class
# simple_nn = SimpleNeuralNetwork(input_dim, hidden_units, output_dim)
simple_nn = SimpleNeuralNetwork(input_dim, output_dim)

In [10]:
simple_nn.tune_hyperparameters(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_val, y_val))

In [11]:
simple_nn.tune_hyperparameters(train_features, train_labels, epochs=30, batch_size=64, validation_data=(X_val, y_val))

In [12]:
# Train the model
simple_nn.train(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_val, y_val))

# Evaluate the model
simple_nn.evaluate(X_val, y_val)

In [13]:
simple_nn.train(train_features, train_labels, epochs=10, batch_size=64)

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Custom RBF activation function
def rbf_activation(x):
    return tf.math.exp(-1.0 * tf.square(x))

class SimpleNeuralNetwork:
    def __init__(self, input_dim, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.model = None  # Initialize model attribute

    def build_model(self, hp):
        model = Sequential()

        # Add the specified number of hidden layers with units and activations
        for i in range(hp.get('num_hidden_layers')):
            model.add(Dense(
                units=hp.get(f'units_{i}'),
                activation=hp.get(f'activation_{i}'),
                input_dim=self.input_dim if i == 0 else None
            ))

        model.add(Dense(units=self.output_dim, activation='sigmoid'))

        # Compile the model with the specified learning rate
        model.compile(
            loss='binary_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=hp.get('learning_rate')),
            metrics=['accuracy']
        )

        self.model = model  # Store the model in the attribute
        return model

    # Other methods remain unchanged
    # def train(self, X_train, y_train, epochs, batch_size, validation_data=None):
    #     class_weights = {0:8, 1:3}  # Adjust class weights based on your data distribution
    #     self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data, class_weight=class_weights)
    def train(self, X_train, y_train, epochs, batch_size, validation_data=None):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    def evaluate(self, X_test, y_test):
        loss, accuracy = self.model.evaluate(X_test, y_test)
        print(f'Test Loss: {loss:.4f}')
        print(f'Test Accuracy: {accuracy:.4f}')

    def predict(self, X_data, threshold=0.5):
        predictions_proba = self.model.predict(X_data)
        predictions = (predictions_proba > threshold).astype(int)
        return predictions


In [15]:
input_dim = X_train.shape[1]
hidden_units = input_dim
output_dim = 1

# Instantiate SimpleNeuralNetwork with input_dim and output_dim
simple_nn = SimpleNeuralNetwork(input_dim, output_dim)

# Get the hyperparameters dictionary
hyperparameters = {'num_hidden_layers': 1,
                   'units_0': 74,
                   'activation_0': 'relu',
                   'learning_rate': 0.0001,
                   'units_1': 116,
                   'activation_1': 'relu',
                   'units_2': 118,
                   'activation_2': 'tanh'}

# Build the model using the provided hyperparameters
model = simple_nn.build_model(hyperparameters)

# Now, you can train, evaluate, and predict with this model

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from kerastuner.tuners import RandomSearch

# Custom RBF activation function
def rbf_activation(x):
    return tf.math.exp(-1.0 * tf.square(x))

class SimpleNeuralNetwork:
    def __init__(self, input_dim, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.model = None  # Initialize model attribute

    def build_model(self, hp):
        model = Sequential()

        # Add the specified number of hidden layers with units and activations
        for i in range(hp.get('num_hidden_layers')):
            model.add(Dense(
                units=hp.get(f'units_{i}'),
                activation=hp.get(f'activation_{i}'),
                kernel_initializer=hp.get(f'kernel_initializer_{i}'),
                kernel_regularizer=hp.get(f'kernel_regularizer_{i}'),
                input_dim=self.input_dim if i == 0 else None
            ))

            if hp.get(f'batch_normalization_{i}'):
                model.add(tf.keras.layers.BatchNormalization())

            model.add(tf.keras.layers.Dropout(hp.get(f'dropout_{i}')))
                    
        model.add(Dense(units=self.output_dim, activation='sigmoid'))

        # Compile the model with the specified learning rate
        model.compile(
            loss='binary_crossentropy',
            optimizer=tf.keras.optimizers.get(hp.get('optimizer')),
            metrics=['accuracy']
        )

        self.model = model  # Store the model in the attribute
        return model

    def tune_hyperparameters(self, X_train, y_train, epochs, batch_size, validation_data=None):
        tuner = RandomSearch(
            self.build_model,
            objective='val_accuracy',
            max_trials=10,  # Number of hyperparameter combinations to try
            directory='keras_tuner_dir',  # Directory to store the results
            project_name='simple_neural_network'
        )

        tuner.search(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

        # Get the best model and its hyperparameters
        best_model = tuner.get_best_models(num_models=1)[0]
        best_hyperparameters = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

        self.model = best_model
        return best_hyperparameters

    
        
    # Other methods remain unchanged
    # def train(self, X_train, y_train, epochs, batch_size, validation_data=None):
    #     class_weights = {0:8, 1:3}  # Adjust class weights based on your data distribution
    #     self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data, class_weight=class_weights)
    def train(self, X_train, y_train, epochs, batch_size, validation_data=None):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    def evaluate(self, X_test, y_test):
        loss, accuracy = self.model.evaluate(X_test, y_test)
        print(f'Test Loss: {loss:.4f}')
        print(f'Test Accuracy: {accuracy:.4f}')

    def predict(self, X_data, threshold=0.5):
        predictions_proba = self.model.predict(X_data)
        predictions = (predictions_proba > threshold).astype(int)
        return predictions

# Usage
input_dim = X_train.shape[1]
output_dim = 1

# Instantiate SimpleNeuralNetwork with input_dim and output_dim
simple_nn = SimpleNeuralNetwork(input_dim, output_dim)

# Get the hyperparameters dictionary
hyperparameters = {'num_hidden_layers': 2,
                   'units_0': 128,
                   'activation_0': 'relu',
                   'kernel_initializer_0': 'he_normal',
                   'kernel_regularizer_0': None,
                   'batch_normalization_0': True,
                   'dropout_0': 0.3,
                   'units_1': 64,
                   'activation_1': 'relu',
                   'kernel_initializer_1': 'glorot_uniform',
                   'kernel_regularizer_1': 'l2',
                   'batch_normalization_1': False,
                   'dropout_1': 0.2,
                   'optimizer': 'adam',
                   'learning_rate': 0.0001}

# Build and train the model using the provided hyperparameters
model = simple_nn.build_model(hyperparameters)
simple_nn.train(train_features, train_labels, epochs=10, batch_size=8192)


In [17]:
simple_nn.train(train_features, train_labels, epochs=3, batch_size=8192)
# epochs and batch size are hyperparameters

Epoch 1/3


22/22 [==============================] - 2s 8ms/step - loss: 104.4791 - accuracy: 0.5000
Epoch 2/3
22/22 [==============================] - 0s 6ms/step - loss: 69.5368 - accuracy: 0.5000
Epoch 3/3
22/22 [==============================] - 0s 5ms/step - loss: 34.7981 - accuracy: 0.5000


In [28]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, concatenate
from tensorflow.keras.optimizers import Adam

num_features=24
# Define hyperparameters
num_hidden_layers = 2
units = [74, 116, 118]
activations = ['relu', 'relu', 'tanh']
# activations = ['relu', 'relu', 'relu']
learning_rate = 0.0001

# Input layer
input_layer = Input(shape=(num_features,))

# Define wide branch
wide_branch = Dense(1)(input_layer)

# Define deep branch
deep_branch = input_layer
for i in range(num_hidden_layers):
    deep_branch = Dense(units[i], activation=activations[i])(deep_branch)

# Concatenate branches
merged = concatenate([wide_branch, deep_branch])

# Output layer
output_layer = Dense(1, activation='sigmoid')(merged)

# Create the model
model1 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model1.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 24)]                 0         []                            
                                                                                                  
 dense_11 (Dense)            (None, 74)                   1850      ['input_3[0][0]']             
                                                                                                  
 dense_10 (Dense)            (None, 1)                    25        ['input_3[0][0]']             
                                                                                                  
 dense_12 (Dense)            (None, 116)                  8700      ['dense_11[0][0]']            
                                                                                            

In [33]:
# Train the model
epochs = 20
batch_size = 4096

model1.fit(train_features, train_labels, epochs=epochs, batch_size=batch_size)

# Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(X_test, y_test)
# print(f'Test Loss: {test_loss:.4f}')
# print(f'Test Accuracy: {test_accuracy:.4f}')

# Make predictions on the test set
predictions_proba1 = pd.DataFrame(model1.predict(test_features))
predictions_proba1.head()
predictions = (predictions_proba1 > 0.65).astype(int)

Epoch 1/20
44/44 [==============================] - 0s 7ms/step - loss: 0.4721 - accuracy: 0.7734
Epoch 2/20
44/44 [==============================] - 0s 6ms/step - loss: 0.4691 - accuracy: 0.7751
Epoch 3/20
44/44 [==============================] - 0s 8ms/step - loss: 0.4692 - accuracy: 0.7749
Epoch 4/20
44/44 [==============================] - 0s 7ms/step - loss: 0.4683 - accuracy: 0.7750
Epoch 5/20
44/44 [==============================] - 0s 7ms/step - loss: 0.4796 - accuracy: 0.7696
Epoch 6/20
44/44 [==============================] - 0s 7ms/step - loss: 0.4763 - accuracy: 0.7709
Epoch 7/20
44/44 [==============================] - 0s 8ms/step - loss: 0.4753 - accuracy: 0.7715
Epoch 8/20
44/44 [==============================] - 0s 8ms/step - loss: 0.4668 - accuracy: 0.7764
Epoch 9/20
44/44 [==============================] - 0s 8ms/step - loss: 0.4684 - accuracy: 0.7752
Epoch 10/20
44/44 [==============================] - 0s 10ms/step - loss: 0.4707 - accuracy: 0.7741
Epoch 11/20
44/44 

In [34]:
# # Make predictions on test data
# threshold = 0.65
# # threshold = 0.65 gives best result
# predictions = simple_nn.predict(test_features, threshold=threshold)

In [35]:
# writing predictions to submission.csv
fin_sub = pd.read_csv("./sample_submissions.csv")
fin_sub["Task"] = predictions
fin_sub.head()

,ID,Task
0,100721,1
1,30234,0
2,28624,0
3,31173,1
4,573,0


In [36]:
fin_sub.to_csv('./submission_NN.csv', encoding='utf-8', index=False)